In [56]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
from tensorflow.keras import layers, models, utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import time
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

In [57]:
def load_images(folder_path, image_size, batch_size=64, test_size=0.2, random_state=42):
    """
    Loads images with some preprocessing into train and test data generators using tensorflow's ImageDataGenerator
    :folder_path: folder location of the images
    :image_size: wanted size after loading
    :batch_size: how many images at once the data generator will generate
    """
    datagen = ImageDataGenerator(rescale=1./255, validation_split=test_size)
    
    data_generator = datagen.flow_from_directory(
        folder_path,
        target_size=image_size,
        batch_size=batch_size,
        class_mode='sparse',
        subset='training'
    )

    validation_data_generator = datagen.flow_from_directory(
        folder_path,
        target_size=image_size,
        batch_size=batch_size,
        class_mode='sparse',
        subset='validation'
    )

    return data_generator, validation_data_generator

In [58]:
# Class implementation for CNN models. Inıtializes a model with given parameters with Tensorflow's Sequential
class CNNClassifier:
    def __init__(self, num_conv_layers=2, num_filters=32, kernel_size=(3, 3), input_shape=(128, 128, 3), num_classes=10):
        self.num_conv_layers = num_conv_layers
        self.num_filters = num_filters
        self.kernel_size = kernel_size
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.model = self.build_model()

    def build_model(self):
        model = models.Sequential()
        model.add(layers.Conv2D(self.num_filters, self.kernel_size, activation='relu', input_shape=self.input_shape))
        model.add(layers.MaxPooling2D((2, 2))) 
        for _ in range(self.num_conv_layers - 1):   # Add wanted num_conv_layers - 1 layer after input layer
            model.add(layers.Conv2D(self.num_filters, self.kernel_size, activation='relu'))
            model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Flatten())
        model.add(layers.Dropout(0.25))
        model.add(layers.Dense(128, activation='relu'))
        model.add(layers.Dense(self.num_classes, activation='softmax'))

        model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',   #sparse has chosen for this specific task
                      metrics=['accuracy'])
        return model

    def train(self, train_data, validation_data, epochs=10, batch_size=64):
        self.history = self.model.fit(train_data, epochs=epochs, batch_size=batch_size, validation_data=validation_data)

    def evaluate(self, test_data, y_true):
        y_pred = self.model.predict_classes(test_data)
        print("\nClassification Report:\n", classification_report(y_true, y_pred))
        print("\nAccuracy:", accuracy_score(y_true, y_pred))
        return y_pred
    
    def summary(self):
        self.model.summary()

    def plot_metrics(self):
        history = self.history

        # Plot training and validation accuracy
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        plt.plot(history.history['accuracy'], label='Training Accuracy', color='blue')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='orange')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.title('Training and Validation Accuracy')
        plt.legend()

        # Plot training and validation loss
        plt.subplot(1, 2, 2)
        plt.plot(history.history['loss'], label='Training Loss', color='blue')
        plt.plot(history.history['val_loss'], label='Validation Loss', color='orange')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Training and Validation Loss')
        plt.legend()

        plt.tight_layout()
        plt.show()

In [69]:
# Create model for each combination of hyperparameter search space, train, save the important result features, return a results dataframe 
def model_comparison(data_folder, img_size_list, num_filters_list, kernel_size_list, num_conv_layers_list):
    results = []
    
    # Choosing each combination by for loops
    for img_size in img_size_list:
        train_data_generator, validation_data_generator = load_images(data_folder, img_size)    # Load the data only once as it has only one tuned hyperparameter
        input_shape = img_size + (3,)
        for num_filters in num_filters_list:
            for kernel_size in kernel_size_list:
                for num_conv_layers in num_conv_layers_list:
                    print(f"\nTraining with the hyperparameters:\nImage size: {img_size}    Number of convolution layers: {num_conv_layers}    Number of filters: {num_filters}   Kernel size: {kernel_size}")
                    
                    model = CNNClassifier(num_conv_layers=num_conv_layers, num_filters=num_filters, kernel_size=kernel_size, input_shape=input_shape)
                    
                    # Calculate training time for comparison
                    start_time = time.time()
                    model.train(train_data_generator, validation_data_generator)
                    end_time = time.time()
                    training_time = round(end_time - start_time, 1)     # Rounds it to 1 digit sensitivity for readibility
                    
                    # Takes the last epoch's value and rounds it to 3 digit sensitivity for readibility
                    train_acc = round(model.history.history['accuracy'][-1], 3)
                    test_acc = round(model.history.history['val_accuracy'][-1], 3)
                    train_loss = round(model.history.history['loss'][-1], 3)
                    test_loss = round(model.history.history['val_loss'][-1], 3)

                    # Saves result in a dictionary
                    result = {
                        'img_size': img_size,
                        'num_conv_layers': num_conv_layers,
                        'num_filters': num_filters,
                        'kernel_size': kernel_size,
                        'train accuracy': train_acc,
                        'test accuracy': test_acc,
                        'train loss': train_loss,
                        'test_loss': test_loss,
                        'training time': training_time
                    }
                    results.append(result)  # Merges all model's result
                    del model   # Deletes the last model for less memory usage
                    
    return pd.DataFrame(results)    # Returns as dataframe for readibility

In [70]:
# Path of folder where the data is 
data_folder = "C:\\20+\\bitirme\\spectrograms\\stft"

In [71]:
# Hyperparameter search space, each of them must be a list of proper hyperparameters
img_size_list = [(256, 128), (256, 256), (128, 128)]
num_filters_list = [32, 64]
kernel_size_list = [(3, 3), (5, 5)]
num_conv_layers_list = [2, 3, 4]


In [72]:
# Call the comparison function with the lists and the path. Takes around 1.5 hr 
results = model_comparison(data_folder, img_size_list=img_size_list, num_filters_list=num_filters_list, kernel_size_list=kernel_size_list, num_conv_layers_list=num_conv_layers_list)

Found 800 images belonging to 10 classes.
Found 199 images belonging to 10 classes.

Training with the hyperparameters:
Image size: (256, 128)    Number of convolution layers: 2    Number of filters: 32   Kernel size: (3, 3)
Epoch 1/10
13/13 [==============================] - 10s 763ms/step - loss: 2.9467 - accuracy: 0.1737 - val_loss: 2.1445 - val_accuracy: 0.1307
Epoch 2/10
13/13 [==============================] - 9s 724ms/step - loss: 2.0253 - accuracy: 0.1975 - val_loss: 1.8986 - val_accuracy: 0.1608
Epoch 3/10
13/13 [==============================] - 10s 739ms/step - loss: 1.7562 - accuracy: 0.3638 - val_loss: 1.8190 - val_accuracy: 0.2613
Epoch 4/10
13/13 [==============================] - 9s 722ms/step - loss: 1.5081 - accuracy: 0.4837 - val_loss: 1.8634 - val_accuracy: 0.3819
Epoch 5/10
13/13 [==============================] - 9s 692ms/step - loss: 1.2282 - accuracy: 0.5962 - val_loss: 1.7776 - val_accuracy: 0.3317
Epoch 6/10
13/13 [==============================] - 9s 725ms/st

In [74]:
results

,img_size,num_conv_layers,num_filters,kernel_size,train accuracy,test accuracy,train loss,test_loss,training time
0,"(256, 128)",2,32,"(3, 3)",0.844,0.422,0.493,1.715,94.2
1,"(256, 128)",3,32,"(3, 3)",0.642,0.472,0.985,1.564,88.4
2,"(256, 128)",4,32,"(3, 3)",0.519,0.382,1.310,1.726,88.6
3,"(256, 128)",2,32,"(5, 5)",0.734,0.447,0.804,1.812,103.1
4,"(256, 128)",3,32,"(5, 5)",0.585,0.407,1.146,1.785,98.9
5,"(256, 128)",4,32,"(5, 5)",0.494,0.322,1.421,1.742,102.5
6,"(256, 128)",2,64,"(3, 3)",0.868,0.407,0.422,2.356,141.2
7,"(256, 128)",3,64,"(3, 3)",0.611,0.432,1.073,1.997,133.0
8,"(256, 128)",4,64,"(3, 3)",0.525,0.442,1.307,1.689,130.9
9,"(256, 128)",2,64,"(5, 5)",0.540,0.352,1.293,1.915,190.2


In [73]:
# Write results as an excel file. Saves in same folder as default. Change the variable where2Write if needed. 
where2Write = 'cnn_stft'
results.to_excel(where2Write + '.xlsx', index=False)

In [104]:
# Function that calculates impacts of each parameter on the test accuracy. Uses a correlation matrix to do so. 
def calculate_impacts(df):
    df_copy = df.copy()    # Works on a copy not to be 'Pythoned'

    # Function for mapping img size to integer values as they are not
    def map_img_size(value):
        if value == "(256, 128)":
            return 1.5
        elif value == "(256, 256)":
            return 2
        elif value == "(128, 128)":
            return 1

    # Apply map function on the corresponded column.    
    df_copy['img_size'] = df_copy['img_size'].apply(lambda x: map_img_size(x))

    # Same operations above for kernel_size
    def map_kernel_size(value):
        if value == "(3, 3)":
            return 3
        elif value == "(5, 5)":
            return 5

    df_copy['kernel_size'] = df_copy['kernel_size'].apply(lambda x: map_kernel_size(x))

    # Columns that are going to be calculated for impact
    columns_of_interest = ['img_size', 'num_conv_layers', 'num_filters', 'kernel_size', 'test accuracy']

    # Create a new DataFrame with only the selected columns
    df_selected = df_copy[columns_of_interest]

    # Calculate correlation coefficients
    correlation_matrix = df_selected.corr()

    # Drop test accuracy from corr matrix to see others impact. Round 2 digit sensitivity. 
    impact_on_test_accuracy = correlation_matrix['test accuracy'].drop('test accuracy').round(2)

    # Display the impact of each parameter on test accuracy
    print("Impact on test accuracy:")
    print(impact_on_test_accuracy)
    del df_copy     # Clears memory

In [105]:
calculate_impacts(results)

Impact on test accuracy:
img_size           0.18
num_conv_layers   -0.47
num_filters        0.16
kernel_size       -0.25
Name: test accuracy, dtype: float64
